# AUDL Game Stats Data Collection
 - https://www.docs.audlstats.com/
 - Contents:
   - [Games List](#List-of-Games)
   - [Stats Investigation](#Game-Stats-Investigations)
     - [API - stats summary](#USE-THIS-ONE-TO-GET-STATS)
     - [API - full game stats](#ONCE-PER-TEAM,-USE-THIS-ONE-TO:)
     - [Initial Plan](#Data-Plan)
   - [Collection Loop](#Collect)
   - [dtype conversions](#Datatype-Optimzations-for-Memory-Reduction)
   - [historical team info](#Team-Info-Additions)
   - [save to parquet](#Parquet-Storage)
---
## Try 1 - success
#### should add TZinfo to date
#### add location information to each game
#### keep gameID to reference AUDL site / add URLs
### Team Info summary - double counting all-stars
---
## Try 2 - fix above
#### 

---

In [133]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

## List of Games

In [60]:
URL = 'https://www.backend.audlstats.com/api/v1/games?'
parameters = 'date=2011:2023' # all games

In [61]:
URL = f'{URL}{parameters}'
r = requests.get(URL)
if r.status_code == 200:
    games = r.json()['data']
else:
    print(r.status_code)

In [62]:
gamelist = [val['gameID'] for val in games if val['status'] == 'Final']

In [64]:
len(gamelist)

1493

## Game Stats Investigations

URLS

 - Advanced Stats: `https://theaudl.com/stats/game/2023-05-19-LA-SLC`
   - `https://www.backend.audlstats.com/stats-pages/game/2023-05-19-LA-SLC`
   - **returns all data going into the advanced stats pages, will use to extract team information**
 -    
 - Normal game page: `https://theaudl.com/league/game/2023-05-19-LA-SLC`
   - `https://www.backend.audlstats.com/web-api/game-stats?gameID=2023-05-19-LA-SLC`
   - **returns the game stats I want**

**Investigating pages that are queried by AUDL page**

found one page that has all the information supplied to tables.
 - team stats at very end (with better column names)
 - game information in beginning
 - followed by roster info, then super long string of the position maps?

### USE THIS ONE TO GET THE STATS

In [27]:
URL = 'https://www.backend.audlstats.com/web-api/game-stats?gameID=2023-05-19-LA-SLC'

r = requests.get(URL)
if r.status_code == 200:
    data = r.json()

In [28]:
data

{'awayTeam': 'LA Aviators',
 'homeTeam': 'Salt Lake Shred',
 'leaderCategories': [{'title': 'Assists',
   'home': {'leaders': 3, 'count': 3},
   'away': {'leaders': [{'playerID': 'smcdougal',
      'firstName': 'Sean',
      'lastName': 'McDougall'}],
    'count': 3}},
  {'title': 'Goals',
   'home': {'leaders': [{'playerID': 'jkerr',
      'firstName': 'Jordan',
      'lastName': 'Kerr'}],
    'count': 5},
   'away': {'leaders': [{'playerID': 'mosborne',
      'firstName': 'Marcel',
      'lastName': 'Osborne'}],
    'count': 3}},
  {'title': 'Blocks',
   'home': {'leaders': [{'playerID': 'jhoffman',
      'firstName': 'Jonny',
      'lastName': 'Hoffman'},
     {'playerID': 'tmounga', 'firstName': 'Tony', 'lastName': 'Mounga'}],
    'count': 2},
   'away': {'leaders': 3, 'count': 1}},
  {'title': 'Completions',
   'home': {'leaders': [{'playerID': 'lyorgason',
      'firstName': 'Luke',
      'lastName': 'Yorgason'}],
    'count': 33},
   'away': {'leaders': [{'playerID': 'pjanas',
 

### ONCE PER TEAM, USE THIS ONE TO:
##### Get their team color and team info in a separate table
```
...
```
    
##### Any other game context
 - reg_season, timestamp, week, 

In [2]:
URL = 'https://www.backend.audlstats.com/stats-pages/game/2023-05-19-LA-SLC'

r = requests.get(URL)
if r.status_code == 200:
    data2 = r.json()

In [3]:
r.status_code

200

In [4]:
data2

{'game': {'id': 3101,
  'team_season_id_home': 277,
  'team_season_id_away': 275,
  'status_id': 100,
  'score_home': 25,
  'score_away': 13,
  'live': None,
  'reg_season': True,
  'ignore_game': False,
  'lock': True,
  'start_timestamp': '2023-05-19T19:00:00.000Z',
  'start_timezone': 'MDT',
  'start_time_tbd': False,
  'aw_section': 'week-4',
  'score_times_home': [0,
   27,
   198,
   279,
   348,
   597,
   838,
   862,
   934,
   1017,
   1138,
   1174,
   1357,
   1400,
   1527,
   1672,
   1801,
   1843,
   1964,
   2116,
   2157,
   2194,
   2248,
   2594,
   2725,
   2822],
  'score_times_away': [0,
   116,
   277,
   352,
   662,
   709,
   988,
   1068,
   1304,
   1613,
   2092,
   2221,
   2556,
   2763],
  'ext_game_id': '2023-05-19-LA-SLC',
  'update_timestamp': '2023-05-21T20:13:21.805Z',
  'location_id': 76,
  'ls_game_id': 'e26e8e8e-0095-4d0b-84e3-4bcb1653b7fb',
  'ticket_url': 'https://saltlakeshred.com/products/may-20th-vs-la-aviators',
  'streaming_url': 'https:/

In [37]:
data2['game']['start_timestamp'].replace('.000','')

'2023-05-19T19:00:00Z'

## Data Plan

 1. Run through each year and collect game stats. SCORE NOT GIVEN IN RETURN!!!
   - GAME ID / Home / Away / Home Score / Away Score / . . .
     - all numbers should be integers, will calculate percentages later
     - include winner, loser, margin. will that be easier for model?
 2. During collection, extract team info into a separate table, once per team
 3. **Later** Get "Player stats", `<div class="stats-table">`. Can relate to games with Game ID.
   - get total players for each team with at least one PP
   - some kind of distribution metrics for PP / total points for each player
     - min / median / max ? 
     - fit distribution and save parameters to plot?
       - plot a few and see

In [6]:
# test table for one game

URL = 'https://www.backend.audlstats.com/web-api/game-stats?gameID=2023-05-19-LA-SLC'

r = requests.get(URL)
if r.status_code == 200:
    data = r.json()

## Collect

~~Test for this year, then do everything~~

***NEXT TIME:***

- do timezone collection ~~and datetime conversion~~ within loop
- collect location information

In [44]:
stat_keys =  {'completionsNumer': 'completions',
 'completionsDenom': 'throws',
 'hucksNumer': 'hucks_completed',
 'hucksDenom': 'hucks',
 'blocks': 'blocks',
 'turnovers': 'turnovers',
 'oLineScores': 'o_scores',
 'oLinePoints': 'o_points',
 'oLinePossessions': 'o_possessions',
 'dLineScores': 'd_scores',
 'dLinePoints': 'd_points',
 'dLinePossessions': 'd_possessions',
 'redZoneScores': 'rz_scores',
 'redZonePossessions': 'rz_possessions'}

In [65]:
start = pd.DataFrame(columns = ['game','date','tz','location','week','home','away','home_score','away_score'])
teams = pd.DataFrame(columns = ['abbrev', 'city', 'name', 'primary', 'secondary'])

In [66]:
# LOOP, initialize dataframes above
with tqdm(gamelist) as pbar:
    for gameID in pbar:
        pbar.set_postfix(game=gameID)
        # complete information
        URL = f'https://www.backend.audlstats.com/stats-pages/game/{gameID}'
        r = requests.get(URL)
        if r.status_code == 200:
            data = r.json().get('game')
        else:
            print(r.status_code, 'for', gameID)
            continue
        if not data:
            print('no data for', gameID)
            continue

        # basic game stats
        gamekey = data['id']
        start.loc[gamekey,'game'] = gameID
        start.loc[gamekey,'date'] = data['start_timestamp']
        start.loc[gamekey,'tz'] = data['start_timezone']
        start.loc[gamekey,'location'] = data['location_id']
        start.loc[gamekey,'week'] = data['aw_section']
        start.loc[gamekey,'home'] = data['team_season_home']['team']['name']
        start.loc[gamekey,'away'] = data['team_season_away']['team']['name']
        start.loc[gamekey,'home_score'] = data['score_home']
        start.loc[gamekey,'away_score'] = data['score_away']

        # collect team information if not already done
        for val in ['team_season_home', 'team_season_away']:
            teamkey = data[val]['team']['id']
            if teamkey not in teams.index:
                teams.loc[teamkey, 'abbrev'] = data[val]['abbrev']
                teams.loc[teamkey, 'city'] = data[val]['city']
                teams.loc[teamkey, 'name'] = data[val]['team'].get('name')
                # colors not provided for All-Stars teams
                teams.loc[teamkey, 'primary'] = data[val]['team'].get('primary_hex')
                teams.loc[teamkey, 'secondary'] = data[val]['team'].get('secondary_hex')

        # detailed home and away stats
        for val in ['tsgAway', 'tsgHome']:
            data = r.json().get(val)
            for stat in stat_keys.keys():
                start.loc[gamekey, f'{val[3:].lower()}_{stat_keys[stat]}'] = data[stat]

100%|██████████| 1493/1493 [05:58<00:00,  4.16it/s, game=2022-07-30-IND-ATL]   


In [71]:
start.head().T

,2492,3041,3101,2658,2460
game,2019-allstars,2023-05-06-DAL-ATX,2023-05-19-LA-SLC,2021-07-11-MIN-CHI,2019-06-22-SD-SEA
date,2019-06-08T18:00:00.000Z,2023-05-06T19:00:00.000Z,2023-05-19T19:00:00.000Z,2021-07-11T14:00:00.000Z,2019-06-22T18:00:00.000Z
tz,CDT,CDT,MDT,CDT,PDT
location,None,92,76,9,62
week,week-10,week-2,week-4,week-6,week-12
home,All-Stars,Sol,Shred,Union,Cascades
away,All-Stars,Legion,Aviators,Wind Chill,Growlers
home_score,28,31,25,18,25
away_score,27,17,13,20,30
away_completions,0.0,254.0,268.0,334.0,257.0


### Datatype Optimzations for Memory Reduction

In [74]:
start.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1493 entries, 2492 to 2886
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   game                  1493 non-null   object 
 1   date                  1493 non-null   object 
 2   tz                    1493 non-null   object 
 3   location              623 non-null    object 
 4   week                  1493 non-null   object 
 5   home                  1493 non-null   object 
 6   away                  1493 non-null   object 
 7   home_score            1493 non-null   object 
 8   away_score            1493 non-null   object 
 9   away_completions      1493 non-null   float64
 10  away_throws           1493 non-null   float64
 11  away_hucks_completed  1493 non-null   float64
 12  away_hucks            1493 non-null   float64
 13  away_blocks           1493 non-null   float64
 14  away_turnovers        1493 non-null   float64
 15  away_o_scores     

In [75]:
# all floats to integers, 63% original size
for col in start.columns[7:]:
    start[col] = start[col].astype(int)

In [76]:
start.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1493 entries, 2492 to 2886
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   game                  1493 non-null   object
 1   date                  1493 non-null   object
 2   tz                    1493 non-null   object
 3   location              623 non-null    object
 4   week                  1493 non-null   object
 5   home                  1493 non-null   object
 6   away                  1493 non-null   object
 7   home_score            1493 non-null   int32 
 8   away_score            1493 non-null   int32 
 9   away_completions      1493 non-null   int32 
 10  away_throws           1493 non-null   int32 
 11  away_hucks_completed  1493 non-null   int32 
 12  away_hucks            1493 non-null   int32 
 13  away_blocks           1493 non-null   int32 
 14  away_turnovers        1493 non-null   int32 
 15  away_o_scores         1493 non-null

In [78]:
# date from object to datetime, no memory reduction
start['date'] = pd.to_datetime(start.date)

In [77]:
# simplify week names, no memory reduction
start.week.unique()

array(['week-10', 'week-2', 'week-4', 'week-6', 'week-12', 'week-7',
       'week-8', 'week-allstars', 'week-3', 'week-1',
       'championship-weekend', 'week-11', 'week-5', 'playoffs', 'week-15',
       'week-13', 'week-14', 'week-9', 'week-17', 'week-16'], dtype=object)

In [79]:
start['week'] = start.week.str.replace('week-','').replace('-weekend','')

In [80]:
start.week.unique()

array(['10', '2', '4', '6', '12', '7', '8', 'allstars', '3', '1',
       'championship-weekend', '11', '5', 'playoffs', '15', '13', '14',
       '9', '17', '16'], dtype=object)

In [100]:
start.columns[2:7]

Index(['tz', 'location', 'week', 'home', 'away'], dtype='object')

In [95]:
# convert team columns and week to category. 57% original size
# can do 2-7
for col in start.columns[4:7]:
    start[col] = start[col].astype('category')

In [101]:
# logical sorting
start = start.sort_values('date')

## Team Info Additions
#### add historical info to teams table (years active, games played)

***DOUBLE COUNTING ALL-STARS***

In [103]:
teams.head()

,abbrev,city,name,primary,secondary
36,KPS,KPS',All-Stars,None,None
35,Rowan,Rowan's,All-Stars,None,None
2,ATX,Austin,Sol,#2B3283,#E74A37
4,DAL,Dallas,Legion,#F0B71A,#000000
40,SLC,Salt Lake,Shred,#00477B,#FFAA00


In [113]:
start[start.home == 'All-Stars']

,game,date,tz,location,week,home,away,home_score,away_score,away_completions,...,home_blocks,home_turnovers,home_o_scores,home_o_points,home_o_possessions,home_d_scores,home_d_points,home_d_possessions,home_rz_scores,home_rz_possessions
2492,2019-allstars,2019-06-08 18:00:00+00:00,CDT,NaN,10,All-Stars,All-Stars,28,27,0,...,0,0,0,0,0,0,0,0,0,0
2964,2022-11-12-allstar-game,2022-11-12 16:00:00+00:00,PST,NaN,allstars,All-Stars,All-Stars,24,42,241,...,12,30,19,34,41,4,23,12,12,14


### All Stars Games
 - 2964: [game stats](https://theaudl.com/stats/game/2022-11-12-allstar-game), [game page](https://theaudl.com/league/game/2022-11-12-allstar-game)
 - 2492: no advanced stats, [game page](https://theaudl.com/league/game/2019-allstars)
 
Manual fixes. 
  - Home/Away for 2492 is KPS's/Rowan's.
  - Home/Away for 2964 is South-West/East-Central
  - .
  - add SW/EC All-Stars to Team-Info with bogus IDs

In [122]:
start.columns[5:7]

Index(['home', 'away'], dtype='object')

In [125]:
# convert back and forth from category

for col in start.columns[5:7]:
    start[col] = start[col].astype(str)

In [124]:
start.loc[2492,'home'] = "KPS's All-Stars"
start.loc[2492,'away'] = "Rowan's All-Stars"
teams.loc[35,'name'] = "Rowan's All-Stars"
teams.loc[36,'name'] = "KPS's All-Stars"

start.loc[2964,'home'] = "SW All-Stars"
start.loc[2964,'away'] = "EC All-Stars"

teams.loc[98,'abbrev'] = 'SW'
teams.loc[98,'city'] = 'South-West'
teams.loc[98,'name'] = 'SW All-Stars'
teams.loc[99,'abbrev'] = 'EC'
teams.loc[99,'city'] = 'East-Central'
teams.loc[99,'name'] = 'EC All-Stars'

In [ ]:
# convert back and forth from category

for col in start.columns[5:7]:
    start[col] = start[col].astype(category)

## Add Historical Info

In [126]:
games_played = pd.Series(start.value_counts('home') + start.value_counts('away'))

In [143]:
# add games played for each team, years active, W/L/T
games_played = pd.concat([pd.DataFrame(start.value_counts('home')).T, pd.DataFrame(start.value_counts('away')).T]).fillna(0)
#games_played.drop(index='All-Stars', inplace=True)
games_played

,AlleyCats,Empire,Rush,Radicals,Mechanix,Breeze,Union,Wind Chill,Phoenix,Spiders,...,Cranes,Hammerheads,Nitro,Lions,Rampage,Havoc,SW All-Stars,KPS's All-Stars,EC All-Stars,Rowan's All-Stars
0,74,74,72,72,70,68,66,64,62,60,...,8,8,8,7,6,2,1.0,1.0,0.0,0.0
0,76,71,66,67,69,68,66,66,66,57,...,8,8,8,7,9,2,0.0,0.0,1.0,1.0


In [145]:
games_played.sum().index

Index(['AlleyCats', 'Empire', 'Rush', 'Radicals', 'Mechanix', 'Breeze',
       'Union', 'Wind Chill', 'Phoenix', 'Spiders', 'Cascades', 'Flyers',
       'Royal', 'Aviators', 'Growlers', 'Thunderbirds', 'Hustle', 'Legion',
       'Cannons', 'Outlaws', 'Sol', 'FlameThrowers', 'Revolution', 'Dragons',
       'Nightwatch', 'Riptide', 'Express', 'Glory', 'Summit', 'Spinners',
       'Shred', 'Constitution', 'Cranes', 'Hammerheads', 'Nitro', 'Lions',
       'Rampage', 'Havoc', 'SW All-Stars', 'KPS's All-Stars', 'EC All-Stars',
       'Rowan's All-Stars'],
      dtype='object')

In [146]:
games_played = games_played.sum()

for team in games_played.index:
    ind = teams[teams.name == team].index
    teams.loc[ind, 'games'] = games_played[team]

    teams.loc[ind, 'first'] = start[(start.home == team) | (start.away == team)].date.min().year
    teams.loc[ind, 'last'] = start[(start.home == team) | (start.away == team)].date.max().year
    
    teams.loc[ind, 'wins'] = start[(start.home == team) & (start.home_score > start.away_score) | (start.away == team) & (start.home_score < start.away_score)].shape[0]
    teams.loc[ind, 'losses'] = start[(start.home == team) & (start.home_score < start.away_score) | (start.away == team) & (start.home_score > start.away_score)].shape[0]
    teams.loc[ind, 'ties'] = start[(start.home == team) & (start.home_score == start.away_score) | (start.away == team) & (start.home_score == start.away_score)].shape[0]
    

In [147]:
# convert dtype to string
teams = teams.convert_dtypes()

In [152]:
# I may have lost some colors . . not sure how
first_teams =  pd.read_parquet('data/try 1/team_info.parquet')

In [175]:
for ind in first_teams.index:
    if type(first_teams.loc[ind,'primary']) == str:
        teams.loc[ind,'primary'] = first_teams.loc[ind,'primary']
    if type(first_teams.loc[ind,'secondary']) == str:
        teams.loc[ind,'secondary'] = first_teams.loc[ind,'secondary']

In [177]:
teams.sort_index()

,abbrev,city,name,primary,secondary,games,first,last,wins,losses,ties
1,ATL,Atlanta,Hustle,#333366,#666666,99,2015,2023,55,44,0
2,ATX,Austin,Sol,#2B3283,#E74A37,85,2016,2023,38,47,0
3,CHI,Chicago,Union,#002D72,#41B6E6,132,2013,2023,74,57,1
4,DAL,Dallas,Legion,#F0B71A,#000000,93,2016,2023,64,29,0
5,DC,DC,Breeze,#0A3751,#B7202B,136,2013,2023,82,53,1
6,DET,Detroit,Mechanix,#660000,#000000,139,2012,2023,16,123,0
7,IND,Indianapolis,AlleyCats,#00703C,#000000,150,2012,2023,79,71,0
8,NSH,Nashville,Nightwatch,<NA>,<NA>,56,2015,2018,8,48,0
9,TB,Tampa Bay,Cannons,<NA>,<NA>,94,2015,2022,36,58,0
10,LA,LA,Aviators,#A22B2A,#000000,102,2015,2023,55,47,0


## Parquet Storage
### notes from previous work . . .
#### arrow does not support float16, but fastparquet doesn't maintain categories
#### compression sizes brotli --> gzip --> snappy --> None

In [178]:
teams.to_parquet('data/team_info.parquet', engine='pyarrow', compression='gzip')

In [102]:
start.to_parquet('data/game_stats.parquet', engine='pyarrow', compression='gzip')

## Exploration

In [201]:
start.head()

,date,week,home,away,home_score,away_score,away_completions,away_throws,away_hucks_completed,away_hucks,...,home_blocks,home_turnovers,home_o_scores,home_o_points,home_o_possessions,home_d_scores,home_d_points,home_d_possessions,home_rz_scores,home_rz_possessions
1891,2012-04-14 19:00:00+00:00,1,Revolution,Mechanix,19,17,160,0,0,0,...,11,34,0,0,0,0,0,0,0,0
1894,2012-04-14 19:00:00+00:00,1,Rampage,Constitution,23,29,162,352,0,0,...,9,28,0,0,0,0,0,0,0,0
1893,2012-04-14 19:00:00+00:00,1,Spinners,Dragons,26,14,180,190,0,0,...,18,24,0,0,0,0,0,0,0,0
1892,2012-04-14 19:00:00+00:00,1,Cranes,AlleyCats,22,21,176,189,0,0,...,10,21,0,0,0,0,0,0,0,0
1897,2012-04-21 19:00:00+00:00,2,Spinners,Constitution,22,16,197,203,0,0,...,13,23,0,0,0,0,0,0,0,0


In [210]:
team = 'Spiders'
start[(start.date.dt.year == 2015) & ((start.home == team) & (start.home_score > start.away_score) | (start.away == team) & (start.home_score < start.away_score))]

,date,week,home,away,home_score,away_score,away_completions,away_throws,away_hucks_completed,away_hucks,...,home_blocks,home_turnovers,home_o_scores,home_o_points,home_o_possessions,home_d_scores,home_d_points,home_d_possessions,home_rz_scores,home_rz_possessions
1884,2015-04-12 13:00:00+00:00,1,Spiders,Riptide,32,26,246,275,0,0,...,10,22,22,28,33,10,34,22,0,0
1878,2015-04-18 19:00:00+00:00,2,Spiders,FlameThrowers,29,26,347,366,0,0,...,9,16,22,28,35,7,31,12,0,0
1860,2015-04-26 13:00:00+00:00,3,Spiders,Growlers,31,18,289,321,0,0,...,13,20,19,20,24,12,33,31,0,0
1856,2015-05-02 19:00:00+00:00,4,Growlers,Spiders,21,23,312,331,0,0,...,12,23,16,25,30,5,23,16,0,0
1848,2015-05-03 13:30:00+00:00,4,Aviators,Spiders,19,20,254,279,0,0,...,8,28,12,22,32,7,21,17,0,0
1841,2015-05-09 18:00:00+00:00,5,FlameThrowers,Spiders,18,20,248,277,0,0,...,11,31,13,22,30,5,20,21,0,0
1830,2015-05-15 19:30:00+00:00,6,Spiders,FlameThrowers,20,15,267,299,0,0,...,13,27,13,17,25,7,21,24,0,0
1803,2015-05-30 19:00:00+00:00,8,Spiders,Aviators,27,19,277,303,0,0,...,12,17,16,21,25,11,29,22,0,0
1778,2015-06-13 18:00:00+00:00,10,Riptide,Spiders,23,26,234,255,0,0,...,16,25,17,28,38,6,25,10,0,0
1768,2015-06-20 19:00:00+00:00,11,Spiders,Cascades,35,28,330,351,0,0,...,13,13,25,29,35,10,37,16,0,0


In [203]:
teams.sort_values('games', ascending=False)

,abbrev,city,name,primary,secondary,games,first,last,wins,losses,ties
7,IND,Indianapolis,AlleyCats,#00703C,#000000,150,2012,2023,79,71,0
14,NY,New York,Empire,#00FF00,#231F20,145,2013,2023,104,41,0
6,DET,Detroit,Mechanix,#660000,#000000,139,2012,2023,16,123,0
11,MAD,Madison,Radicals,#003A5C,#ACA300,139,2013,2023,105,34,0
23,TOR,Toronto,Rush,#C52033,#000000,138,2013,2023,102,36,0
5,DC,DC,Breeze,#0A3751,#B7202B,136,2013,2023,82,53,1
3,CHI,Chicago,Union,#002D72,#41B6E6,132,2013,2023,74,57,1
12,MIN,Minnesota,Wind Chill,#6F7F98,#000000,130,2013,2023,74,55,1
16,PHI,Philadelphia,Phoenix,#A51209,#FD1D14,128,2013,2023,34,93,1
22,SEA,Seattle,Cascades,#08192D,#195187,117,2014,2023,43,74,0
